#### Zoo Model

Descrição: O dataset contém diversas informações sobre animais que devem ser classificados em 7 classes ("Mammal",
    Bird,
    Reptile,
    Fish,
    Amphibian,
    Insect,
    Invertebrate) para 101 instâncias.

Resultados: O modelo se saiu bem em todas as métricas avaliadas, no entanto o dataset é pequeno demais e pode ser que haja overfitting com o modelo.

In [6]:

import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import wisardpkg as wp


In [7]:
# --- 1. Carregar e Inspecionar o Dataset ---
print("--- 1. Carregando e Inspecionando o Dataset Zoo ---")
zoo_model = fetch_ucirepo(id=111)
X_original = zoo_model.data.features
y_original = zoo_model.data.targets.iloc[:, 0]


print(f"Dimensão inicial de X: {X_original.shape}")
print(f"Colunas de X e seus tipos de dados:\n{X_original.dtypes}")
print(f"\nPrimeiras 5 linhas de X:\n{X_original.head()}")
print(f"\nClasses de y:\n{y_original.value_counts()}")
print("-" * 50)

--- 1. Carregando e Inspecionando o Dataset Zoo ---
Dimensão inicial de X: (101, 16)
Colunas de X e seus tipos de dados:
hair        int64
feathers    int64
eggs        int64
milk        int64
airborne    int64
aquatic     int64
predator    int64
toothed     int64
backbone    int64
breathes    int64
venomous    int64
fins        int64
legs        int64
tail        int64
domestic    int64
catsize     int64
dtype: object

Primeiras 5 linhas de X:
   hair  feathers  eggs  milk  airborne  aquatic  predator  toothed  backbone  \
0     1         0     0     1         0        0         1        1         1   
1     1         0     0     1         0        0         0        1         1   
2     0         0     1     0         0        1         1        1         1   
3     1         0     0     1         0        0         1        1         1   
4     1         0     0     1         0        0         1        1         1   

   breathes  venomous  fins  legs  tail  domestic  catsize  
0  

In [8]:
# --- 2. Pré-processamento: One-Hot Encoding para Atributos Categóricos ---
print("--- 2. Pré-processamento: One-Hot Encoding ---")

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False retorna array denso
# sparse_output=False é o mesmo que chamar .toarray() depois.
# Não precisamos do .astype(int).tolist() ainda, faremos a conversão de tipo para np.uint8 depois do split.
X_encoded_dense = encoder.fit_transform(X_original)

print(f"Dimensão de X após One-Hot Encoding: {X_encoded_dense.shape}")
print(f"Tipo dos elementos após One-Hot Encoding: {X_encoded_dense.dtype}")
print(f"Primeiras 2 linhas de X_encoded_dense:\n{X_encoded_dense[:2]}")
print("-" * 50)

--- 2. Pré-processamento: One-Hot Encoding ---
Dimensão de X após One-Hot Encoding: (101, 36)
Tipo dos elementos após One-Hot Encoding: float64
Primeiras 2 linhas de X_encoded_dense:
[[0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.
  0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 0.
  0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1.]]
--------------------------------------------------


In [9]:
# --- 3. Divisão dos Dados em Treino e Teste ---
print("--- 3. Dividindo os Dados em Treino e Teste ---")

# Para o WiSARD, os dados de entrada X devem ser do tipo np.uint8.
# E os rótulos y devem ser strings.
# Fazemos a conversão de tipo após o split para manter a consistência do tipo de dados.
X_encoded_uint8 = X_encoded_dense.astype(np.uint8)
y_str = y_original.astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded_uint8, y_str, test_size=0.7, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")
print("-" * 50)

--- 3. Dividindo os Dados em Treino e Teste ---
Tamanho do conjunto de treino (X_train): 30 amostras, 36 bits
Tamanho do conjunto de teste (X_test): 71 amostras, 36 bits
Tipo dos elementos de X_train: uint8
--------------------------------------------------


In [10]:
# --- 4. Configuração e Treinamento do Modelo WiSARD ---
print("--- 4. Configurando e Treinando o Modelo WiSARD ---")

# addressSize (tuple_size) para o WiSARD.
# O número de bits de entrada de X_encoded_dense para Car Evaluation é 21.
# Um addressSize de 21 ainda é muito grande.
# Escolha um addressSize menor, como 2, 3 ou 4.
# Por exemplo, se X_encoded_dense tem 21 bits, e addressSize=3, teremos 21/3 = 7 RAMs por classe.
addressSize = 3 # Ajuste este valor. 2 ou 3 são bons pontos de partida.

wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
print("Treinando o WiSARD...")
wsd.train(X_train, y_train)
print("Treinamento concluído.")
print("-" * 50)


# --- 5. Classificação e Avaliação do Modelo WiSARD ---
print("--- 5. Classificação e Avaliação do Modelo WiSARD ---")

print("Classificando os dados de teste...")
predictions_str = wsd.classify(X_test)
print("Classificação concluída.")

# Convertendo as previsões de string de volta para um array NumPy para fácil comparação
predictions_np = np.array(predictions_str)
y_test_np = np.array(y_test) # Converte o y_test também para numpy array para comparação elemento a elemento

accuracy_wisard = np.mean(predictions_np == y_test_np)

print("Relatório de Classificação:")
print(classification_report(
    y_test_np, predictions_np,
    labels=np.unique(y_test),
))

--- 4. Configurando e Treinando o Modelo WiSARD ---
WiSARD inicializado com addressSize=3
Treinando o WiSARD...
Treinamento concluído.
--------------------------------------------------
--- 5. Classificação e Avaliação do Modelo WiSARD ---
Classificando os dados de teste...
Classificação concluída.
Relatório de Classificação:
training 30 of 30
c              precision    recall  f1-score   support

           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00        14
           3       1.00      0.67      0.80         3
           4       0.82      1.00      0.90         9
           5       1.00      0.67      0.80         3
           6       1.00      0.67      0.80         6
           7       0.78      1.00      0.88         7

    accuracy                           0.94        71
   macro avg       0.94      0.86      0.88        71
weighted avg       0.96      0.94      0.94        71

classifying 71 of 71
